In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingClassifier

import pandas as pd
import random
import os
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor

import xgboost as xgb
import random

In [ ]:
import jinja2
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

from pycaret.classification import * 

In [ ]:
train = pd.read_excel('./dataset_0914.xlsx',engine ='openpyxl')
# test = label_encoder(test, make_label_map(test))
train = train.drop('number', axis=1)
train.head()

In [ ]:
def check_missing_col(dataframe):
    missing_col = []
    for col in dataframe.columns:
        missing_values = sum(dataframe[col].isna())
        is_missing = True if missing_values >= 1 else False
        if is_missing:
            print(f'결측치가 있는 컬럼은: {col} 입니다')
            print(f'해당 컬럼에 총 {missing_values} 개의 결측치가 존재합니다.')
            missing_col.append([col, dataframe[col].dtype])
    if missing_col == []:
        print('결측치가 존재하지 않습니다')
    return missing_col

missing_col = check_missing_col(train)

# missing_col_test = check_missing_col(test)


In [ ]:
# 결측치를 처리하는 함수를 작성합니다.
def handle_na(data, missing_col):
    temp = data.copy()
    for col, dtype in missing_col:
        if dtype == 'O':
            # 범주형 feature가 결측치인 경우 해당 행들을 삭제해 주었습니다.
            temp = temp.dropna(subset=[col])
    return temp

train = handle_na(train, missing_col)

# 결측치 처리가 잘 되었는지 확인해 줍니다.
missing_col = check_missing_col(train) 

In [ ]:
from sklearn.model_selection import train_test_split

train = train.drop('Mortality', axis = 1).copy() #필요없는 id열 삭제
# train = train.drop('modified_GIB', axis = 1).copy() #필요없는 id열 삭제
# train = train.drop('GIB', axis = 1).copy() #필요없는 id열 삭제
train = train.drop('Intracranial hemorrhage', axis = 1).copy() #필요없는 id열 삭제
train = train.drop('D6', axis = 1).copy() #필요없는 id열 삭제
# train = train.drop('Transfusion', axis = 1).copy() #필요없는 id열 삭제





# train = train.drop('Lab-Creatinine', axis = 1).copy() #필요없는 id열 삭제
# train = train.drop('Lab-Albumin', axis = 1).copy() #필요없는 id열 삭제
# train = train.drop('Lab-Hemoglobin', axis = 1).copy() #필요없는 id열 삭제
# train = train.drop('Lab-INR', axis = 1).copy() #필요없는 id열 삭제

In [ ]:
train

In [ ]:
#### test폴더 따로 있으면 사용
#train_data, val_data = train_test_split(data, test_size=0.5)
# train_data = data
# val_data = test
# train_data.reset_index(inplace=True) #전처리 과정에서 데이터가 뒤섞이지 않도록 인덱스를 초기화
# val_data.reset_index(inplace=True)


#### default
# train.modified_GIB_event = train.modified_GIB_event.replace({1:1, 2:0})
train.GIB = train.GIB.replace({1:1, 2:0})

train_dataset, test_dataset = train_test_split(train, test_size=0.3,shuffle=True,random_state=15)
train_data = train_dataset
test_data = test_dataset

train_data.reset_index(inplace=True) #전처리 과정에서 데이터가 뒤섞이지 않도록 인덱스를 초기화
test_data.reset_index(inplace=True)

train_data = train_data.drop('index', axis = 1).copy() #필요없는 id열 삭제
test_data = test_data.drop('index', axis = 1).copy() #필요없는 id열 삭제

In [ ]:
print( 'train 데이터 셋 모양 :', train_data.shape)
print( 'test 데이터 셋 모양 :', test_data.shape)

# print(train_data.modified_GIB_event.value_counts(), test_data.modified_GIB_event.value_counts())
print(train_data.GIB.value_counts(), test_data.GIB.value_counts())

In [ ]:
# clf = setup(data = train_data, target = 'modified_GIB_event',normalize = True ,session_id=123,silent=True)
clf = setup(data = train_data, target = 'GIB',normalize = True ,session_id=123,silent=True,use_gpu = False )

In [ ]:
compare_models(sort = 'AUC')

In [ ]:
# 모델 생성하기
ada = create_model('et', cross_validation = False)

In [ ]:
gbc = create_model('gbc', cross_validation = False)

In [ ]:
lr = create_model('lr', cross_validation = False)

In [ ]:
# 하이퍼파라미터 튜닝
tuned_ada = tune_model(ada,optimize='AUC', n_iter = 200)

In [ ]:
tuned_gbc = tune_model(gbc,optimize='AUC', n_iter = 200)

In [ ]:
tuned_lr = tune_model(lr,optimize='AUC', n_iter = 200)

In [ ]:
blender_specific = blend_models(estimator_list = [tuned_gbc,tuned_ada,tuned_lr], optimize = 'AUC')

final_model = finalize_model(blender_specific)

In [ ]:
pred = predict_model(tuned_ada, data = test_data)

In [ ]:
plt.figure(figsize = (200,9))
plot_model(estimator =tuned_gbc, plot = 'feature')
plt.show()


# AutoML 시작!!

In [ ]:
#### 1. h2o 분석 준비하기 ####
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import time
import h2o
from h2o.automl import H2OAutoML
from h2o.estimators.gbm import H2OGradientBoostingEstimator
%matplotlib inline
h2o.init( min_mem_size= 16,max_mem_size = 16)
h2o.no_progress()
# H2O 클러스터 가동 시간, 시간대, 버전, 버전 수명, 클러스터 이름, 할당 된 하드웨어 리소스 (노드 수, 메모리, 코어 수), 연결 URL, 노출 된 H2O API 확장 및 사용 된 파이썬 버전과 같은 추가 정보가 출력

In [ ]:
# data_df을 8:2로 나눈다, 50 : 13
train, valid = train_test_split(train_data,
                                test_size=0.2, 
                                shuffle=True)


In [ ]:
max_runtime_secs = 60*60*16

In [ ]:
len(train_dataset), len(train),len(valid),len(test_dataset)

In [ ]:
clf[3]

In [ ]:
%%time
################################################################
## make dataset
# Identify the response and set of predictors
y = "GIB"
x = list(train.columns)  #if x is defined as all columns except the response, then x is not required
x.remove(y)

h2o_train = h2o.H2OFrame(train)
h2o_valid = h2o.H2OFrame(valid)

# For binary classification, response should be a factor
h2o_train[y] = h2o_train[y].asfactor()
h2o_valid[y] = h2o_valid[y].asfactor()

################################################################    
## Run AutoML for 120 seconds
aml = H2OAutoML(max_runtime_secs=max_runtime_secs, exclude_algos =['DRF','GLM'])
# aml = H2OAutoML(max_runtime_secs=max_runtime_secs,max_runtime_secs_per_model = int(max_runtime_secs/10), seed=1)
aml.train(x = x, y = y, training_frame=h2o_train, leaderboard_frame=h2o_valid)
################################################################
## save metric




# Print Leaderboard (ranked by xval metrics)
leaderboard = aml.leaderboard
performance = aml.leader.model_performance(h2o_valid)  # (Optional) Evaluate performance on a test set

model_id   =aml.leader.model_id  # 최고 모델 명
accuracy   =performance.accuracy()  # 정확도
precision  =performance.precision()  # precision
recall     =performance.recall()  # recall
F1         =performance.F1()  # f1
auc        =performance.auc()  # auc
variable_importance=aml.leader.varimp()  # 중요한 입력 변수

print(model_id, accuracy, precision, recall, F1, auc, variable_importance)

In [ ]:
print(performance)

In [ ]:
## 중요 변수 시각화
aml.leader.varimp_plot()

## automl 결과 확인
# Get AutoML object by `project_name`
get_aml = h2o.automl.get_automl(aml.project_name)
# Predict with top model from AutoML Leaderboard on a H2OFrame called 'test'
label_predicted = get_aml.predict(h2o_valid)
print(label_predicted['predict'])
print(h2o_valid['GIB'])

In [ ]:
h2o_test = h2o.H2OFrame(test_data)

# For binary classification, response should be a factor
# h2o_test[y] = h2o_test[y].asfactor()

# Predict with top model from AutoML Leaderboard on a H2OFrame called 'test'
label_predicted = get_aml.predict(h2o_test)
# print(label_predicted['predict'])
# print(h2o_valid['nerdiness'])

In [ ]:
performance = aml.leader.model_performance(h2o_test)
performance

In [ ]:
len(label_predicted)

In [ ]:
h2o_test = h2o.H2OFrame(test_data)

label_predicted = get_aml.predict(h2o_test)
print(label_predicted['predict'])
print(h2o_valid['GIB'])